- 预测2016.11.01-2016.11.14客户流量.结果为非负整数。
- 提供2015.07.01到2016.10.31（除去2015.12.12）的商家数据，用户支付行为数据以及用户浏览行为数据。
- 提供数据的类型统一为string类型，提交预测的类型为整形。文件统一为utf-8编码，没有标题行，并以“,”分隔的csv格式。

### 供参考的特征生成方法

- centrality measures (mostly median) on sales of the same store/weekday/promo during last 3, 6, 12 and 24 months (some spread measures too but they were more prone to overfitting)
- linear model (ridge) predictions for each store on weekday/promo/dayssince2000 on last 3 and 12 months to help the tree model extrapolate
- number of schoolholidays this week, last week and next week
- aggregates by store (avg sales per customer, promo sales ratio, saturday sales ratio, holiday sales ratio)
- daycounters (before, during and after) for things like position within promo cycle, promo2 cycle, summerholidays and refurbishments

External data that were posted to the forum by others (big thanks!):

- maxtemp and precipmm from the weather files by state
- summerholiday start dates


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
data_col=['user_id','shop_id','time_stamp']

In [3]:
shop_info_col=['shop_id','city_name','location_id','per_pay','score','comment_cnt','shop_level','cate_1_name','cate_2_name','cate_3_name']

In [4]:
shop_info=pd.read_csv('../dataset/shop_info.txt',header=None)
shop_info.columns=shop_info_col

shop_info.head()

,shop_id,city_name,location_id,per_pay,score,comment_cnt,shop_level,cate_1_name,cate_2_name,cate_3_name
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果


In [5]:
shop_info.drop(['city_name','cate_1_name','cate_2_name','cate_3_name'],axis=1,inplace=True)

In [6]:
shop_info.head()

,shop_id,location_id,per_pay,score,comment_cnt,shop_level
0,1,885,8,4.0,12.0,2
1,2,64,19,NaN,NaN,1
2,3,774,5,3.0,2.0,0
3,4,380,18,NaN,NaN,1
4,5,263,2,2.0,2.0,0


In [7]:
#data_view=pd.read_csv('../dataset/user_view.txt',header=None)

#data_view.columns=data_col

In [8]:
#data_pay=pd.read_csv('../dataset/user_view.txt',header=None)
#data_pay.columns=data_col

#data_pay.head(15)

In [9]:
#data_pay.time_stamp=pd.to_datetime(data_pay.time_stamp, format='%Y/%m/%d %H:%M:%S')

In [10]:
#data_pay['data']=data_pay.time_stamp.dt.date
#data_pay['day']=data_pay.time_stamp.dt.day
#data_pay['dayofweek']=data_pay.time_stamp.dt.dayofweek
#data_pay['month']=data_pay.time_stamp.dt.month

In [11]:
#data_pay.head()

In [12]:
#feature_daypay=pd.DataFrame(data_pay.groupby(['shop_id','data'])['user_id'].count())
#feature_daypay.rename(columns={'user_id':'daypay'},inplace=True)
#feature_daypay.to_csv('../tempfile/feature_daypay.csv')
feature_daypay=pd.read_csv('../tempfile/feature_daypay.csv')

In [13]:
# 14天的预测值 
# 分割数据集

#  2016-10-18  2016-10-31
#  2016-10-3  2016-10-17

# 过去一周平均业绩
#  过去一个月平均业绩
# 过去3个月平均业绩
# 过去半年平均业绩
# 过去一年平均业绩
# 去年同期平均业绩

In [14]:
feature_daypay.head()

,shop_id,data,daypay
0,1,2015-10-10,188
1,1,2015-10-11,272
2,1,2015-10-12,257
3,1,2015-10-13,239
4,1,2015-10-14,277


In [15]:
train=feature_daypay[feature_daypay.data<'2016-10-18']

test=feature_daypay#[feature_daypay.data>='2016-10-18']

In [16]:
train.shape

(571375, 3)

In [17]:
test.shape

(599275, 3)

In [18]:
train.head()

,shop_id,data,daypay
0,1,2015-10-10,188
1,1,2015-10-11,272
2,1,2015-10-12,257
3,1,2015-10-13,239
4,1,2015-10-14,277


In [19]:
train2=train[train.data>='2016-08-18']
test2=test[test.data>='2016-08-31']

In [20]:
last1_week=train2[(train2.data<'2016-10-18')&(train2.data>='2016-10-11')]
print last1_week['shop_id'].drop_duplicates().count()
last1_week_feature=last1_week.groupby('shop_id')['daypay'].agg({'sum_l1w':np.sum,'mean_l1w':np.mean,'max_l1w':np.max,'min_l1w':np.min,'var_l1w':np.var,'median_l1w':np.median})
last1_week_feature.to_csv('../tempfile/last1_week_feature.csv')
last1_week_feature=pd.read_csv('../tempfile/last1_week_feature.csv')
#last1_week_feature.head()

last2_week=train2[(train2.data<'2016-10-18')&(train2.data>='2016-10-04')]
print last2_week['shop_id'].drop_duplicates().count()
last2_week_feature=last2_week.groupby('shop_id')['daypay'].agg({'sum_l2w':np.sum,'mean_l2w':np.mean,'max_l2w':np.max,'min_l2w':np.min,'var_l2w':np.var,'median_l2w':np.median})
last2_week_feature.to_csv('../tempfile/last2_week_feature.csv')
last2_week_feature=pd.read_csv('../tempfile/last2_week_feature.csv')
#last2_week_feature.head()

last1_month=train2[(train2.data<'2016-10-18')&(train2.data>='2016-09-18')]
print last1_month['shop_id'].drop_duplicates().count()
last1_month_feature=last1_month.groupby('shop_id')['daypay'].agg({'sum_l2m':np.sum,'mean_l2m':np.mean,'max_l2m':np.max,'min_l2m':np.min,'var_l2m':np.var,'median_l2m':np.median})
last1_month_feature.to_csv('../tempfile/last1_month_feature.csv')
last1_month_feature=pd.read_csv('../tempfile/last1_month_feature.csv')
#last1_month_feature.head()

1992
1996
1997


In [21]:
train_feature=shop_info.merge(last1_week_feature,on='shop_id',how='outer')
train_feature=train_feature.merge(last2_week_feature,on='shop_id',how='outer')
train_feature=train_feature.merge(last1_month_feature,on='shop_id',how='outer')

In [22]:
train_feature.shape

(2000, 24)

In [23]:
last1_week=test2[test2.data>='2016-10-25']
print last1_week['shop_id'].drop_duplicates().count()
last1_week_feature=last1_week.groupby('shop_id')['daypay'].agg({'sum_l1w':np.sum,'mean_l1w':np.mean,'max_l1w':np.max,'min_l1w':np.min,'var_l1w':np.var,'median_l1w':np.median})
last1_week_feature.to_csv('../tempfile/last1_week_feature_test.csv')
last1_week_feature=pd.read_csv('../tempfile/last1_week_feature_test.csv')
#last1_week_feature.head()

last2_week=test2[test2.data>='2016-10-18']
print last2_week['shop_id'].drop_duplicates().count()
last2_week_feature=last2_week.groupby('shop_id')['daypay'].agg({'sum_l2w':np.sum,'mean_l2w':np.mean,'max_l2w':np.max,'min_l2w':np.min,'var_l2w':np.var,'median_l2w':np.median})
last2_week_feature.to_csv('../tempfile/last2_week_feature_test.csv')
last2_week_feature=pd.read_csv('../tempfile/last2_week_feature_test.csv')
#last2_week_feature.head()

last1_month=test2[test2.data>='2016-10-01']
print last1_month['shop_id'].drop_duplicates().count()
last1_month_feature=last1_month.groupby('shop_id')['daypay'].agg({'sum_l2m':np.sum,'mean_l2m':np.mean,'max_l2m':np.max,'min_l2m':np.min,'var_l2m':np.var,'median_l2m':np.median})
last1_month_feature.to_csv('../tempfile/last1_month_feature_test.csv')
last1_month_feature=pd.read_csv('../tempfile/last1_month_feature_test.csv')
#last1_month_feature.head()

1998
1999
2000


In [24]:
test_feature=shop_info.merge(last1_week_feature,on='shop_id',how='outer')
test_feature=test_feature.merge(last2_week_feature,on='shop_id',how='outer')
test_feature=test_feature.merge(last1_month_feature,on='shop_id',how='outer')

In [25]:
test_feature.shape

(2000, 24)

In [26]:
train_label=pd.read_csv('../tempfile/train_label_test.csv',header=None)

In [27]:
train_label.shape

(2000, 14)

In [28]:
from xgboost import XGBClassifier,XGBRegressor

In [29]:
clf=XGBRegressor()

In [30]:
test_label=train_label

In [31]:
train_label.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,244,177,207,230,202,243,216,255,172,226,257,251,281,180
1,0,0,0,61,155,143,78,82,97,83,96,96,115,80
2,59,71,88,103,127,125,47,80,61,68,103,102,104,67
3,90,112,59,80,211,221,81,90,104,55,110,215,196,115
4,201,210,221,187,161,240,454,302,1,0,215,203,291,239


In [32]:
test_label.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,244,177,207,230,202,243,216,255,172,226,257,251,281,180
1,0,0,0,61,155,143,78,82,97,83,96,96,115,80
2,59,71,88,103,127,125,47,80,61,68,103,102,104,67
3,90,112,59,80,211,221,81,90,104,55,110,215,196,115
4,201,210,221,187,161,240,454,302,1,0,215,203,291,239


In [33]:
for i in range(0,14):
    print i,
    clf.fit(train_feature,train_label[i])
    test_label[i]=clf.predict(test_feature)

0 1 2 3 4 5 6 7 8 9 10 11 12 13


In [34]:
test_label=test_label.astype(int)

In [35]:
test_label.index=[x for x in range(1,2001)]

In [36]:
test_label.to_csv('../output/pred_xgbR.csv',header=None)

In [37]:
feat_imp=pd.Series(clf.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')

In [106]:
def cal_offline_score(pre_y, real_y):
    """Calculte the offline score according to the score function on Tianchi.

    Args:
        @pre_y: The prediction value
        @real_y: The true value

    Return:
        @L: The offline score of the prediction.

    Raise:
        Error: When the shapes of prediction and real are different.
    """
    score_sum = 0.
    print pre_y.shape
    assert pre_y.shape == real_y.shape
    for shop_index in range(pre_y.shape[0]):
        for day in range(pre_y.shape[1]):
            c_it = float(pre_y[day][shop_index])
            c_itg = float(real_y[day][shop_index])
            # score = (pre_y[shop_index][day]-real_y[shop_index][day])/\
            # (pre_y[shop_index][day]+real_y[shop_index][day])
            if c_itg + c_it == 0:
                c_itg = 1.
                c_it = 1.
            score = abs((c_it - c_itg)/(c_itg + c_it))
            score_sum = score_sum + score
    nT = pre_y.shape[0]*pre_y.shape[1]
    print nT
    print score_sum
    L = score_sum / nT
    return L


In [92]:
best_label=pd.read_csv('../output/0843258.csv',header=None)

In [93]:
best_label.drop(0,axis=1,inplace=True)

In [94]:
best_label.columns=[x for x in range(14)]

In [122]:
test_label=pd.read_csv('../output/pred_xgb.csv',header=None)

In [123]:
test_label.drop(0,axis=1,inplace=True)

In [124]:
test_label.columns=[x for x in range(14)]

In [125]:
best_label.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,218,209,226,242,257,256,205,218,209,226,242,257,256,205
1,91,86,90,98,119,119,88,91,86,90,98,119,119,88
2,69,68,72,87,120,118,71,69,68,72,87,120,118,71
3,90,84,86,118,197,199,98,90,84,86,118,197,199,98
4,222,197,211,215,225,224,205,222,197,211,215,225,224,205


In [126]:
test_label.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,241,220,225,242,244,267,246,241,214,233,257,257,249,223
1,89,75,82,82,123,122,93,86,82,83,86,113,134,85
2,76,83,82,92,99,103,86,86,73,80,97,101,100,86
3,98,85,97,112,159,161,105,103,87,87,118,161,152,102
4,231,187,223,210,241,207,249,245,182,184,245,244,219,245


In [127]:
cal_offline_score(test_label,best_label)

(2000, 14)
28000
1604.11055523


0.05728966268677506

In [130]:
col_label=best_label*0.8+test_label*0.2

In [140]:
for n in range(14):
    col_label[n]=col_label[n].apply(lambda x :round(x))

In [142]:
col_label=col_label.astype(int)

In [144]:
col_label.index=[x for x in range(1,2001)]

In [146]:
col_label.to_csv('../output/col_label.csv',header=None)